## Import libraries

In [54]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

## Load and preprocess data

In [55]:
train_data = np.load('dataset/Shoes - Male - Train.npz')
validation_data = np.load('dataset/Shoes - Male - Validation.npz')
test_data = np.load('dataset/Shoes - Male - Test.npz')

In [56]:
train_data.files

['images', 'labels']

In [57]:
images_train, labels_train = train_data['images'], train_data['labels']
images_val, labels_val = validation_data['images'], validation_data['labels']
images_test, labels_test = test_data['images'], test_data['labels']

In [58]:
images_train.shape

(2073, 120, 90, 3)

In [59]:
labels_train.shape

(2073,)

In [60]:
# Data augmentation
# Flip the image by flip the axis 2
# we have 4 axis now in images (no. of pic, y, x, channel)
# so, if we want to flip horizontally, we flip axis 2

flipped_images_train = np.flip(images_train, axis = 2)
flipped_images_val = np.flip(images_val, axis = 2)
flipped_images_test = np.flip(images_test, axis = 2)

In [61]:
# Don't forget to concatenate labels with itself, cuz we increase pic in images, so labels have to be increased too.
images_train = np.concatenate((images_train, flipped_images_train))
labels_train = np.concatenate((labels_train, labels_train))

images_val = np.concatenate((images_val, flipped_images_val))
labels_val = np.concatenate((labels_val, labels_val))

images_test = np.concatenate((images_test, flipped_images_test))
labels_test = np.concatenate((labels_test, labels_test))

In [62]:
images_val.shape

(518, 120, 90, 3)

In [63]:
labels_val.shape

(518,)

In [64]:
images_test.shape

(518, 120, 90, 3)

In [65]:
labels_test.shape

(518,)

In [66]:
np.unique(labels_train)

array([0, 1, 2, 3, 4])

In [67]:
# Standardize the data
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

## Model

### Define hyperparameters

In [68]:
EPOCHS = 15
BATCH_SIZE = 64

In [69]:
HP_KERNEL_SIZE = hp.HParam('kernel size', hp.Discrete([3,5]))
HP_KERNEL_NUM = hp.HParam('kernel num', hp.Discrete([64,128]))
HP_LAMBDA_REG = hp.HParam('lambda reg', hp.Discrete([1e-5,1e-4,1e-3]))

In [70]:
METRIC = 'accuracy'

with tf.summary.create_file_writer(r'logs/Model 1 (Shoes_Male_L2)/hparam_tuning').as_default():
    hp.hparams_config(
        hparams = [HP_KERNEL_SIZE, HP_KERNEL_NUM, HP_LAMBDA_REG],
        metrics = [hp.Metric(METRIC, display_name = 'accuracy')]
    )

### Model architecture

In [71]:
def train_test_model(hparams, session_num):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE], activation = 'relu', input_shape = (120,90,3), kernel_regularizer = tf.keras.regularizers.L2(hparams[HP_LAMBDA_REG])),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE], activation = 'relu', kernel_regularizer = tf.keras.regularizers.L2(hparams[HP_LAMBDA_REG])),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation = 'relu', kernel_regularizer = tf.keras.regularizers.L2(hparams[HP_LAMBDA_REG])),
        tf.keras.layers.Dense(5, activation = 'softmax')
    ])
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(name = 'sparse_categorical_crossentropy')
    model.compile(optimizer = 'adam', loss = loss_fn, metrics = ['accuracy','sparse_categorical_crossentropy'])
    
    log_dir = "Logs\\Model 1 (Shoes_Male_L2)\\fit\\" + f"run-{session_num}"
    
    # functions for creating confusion matrix
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        return figure

    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""

        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')

        # Closing the figure prevents it from being displayed directly inside the notebook.
        plt.close(figure)

        buf.seek(0)

        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)

        # Add the batch dimension
        image = tf.expand_dims(image, 0)

        return image

    # Define a file writer variable for logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)

        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Boots','Trainers/Sneakers','Sandals/Flip flops/Slippers',
                                                        'Formal shoes', 'Others'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
            
    # callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_sparse_categorical_crossentropy', patience = 2, restore_best_weights = True)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1, profile_batch = 0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end = log_confusion_matrix)
    
    model.fit(images_train,
              labels_train,
              epochs = EPOCHS,
              batch_size = BATCH_SIZE,
              callbacks = [tensorboard_callback, cm_callback, early_stopping],
              validation_data = (images_val, labels_val),
              verbose = 1)
    
    _, accuracy, _ = model.evaluate(images_val, labels_val)
    
    model.save(r'saved_models\Model 1 (Shoes_Male_L2)\Run-{}'.format(session_num))
    
    return accuracy

In [72]:
def run(log_dir, hparams, session_num):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC,accuracy,step=1)

In [73]:
session_num = 1
for kernel_size in HP_KERNEL_SIZE.domain.values:
    for kernel_num in HP_KERNEL_NUM.domain.values:
        for lambda_reg in HP_LAMBDA_REG.domain.values:
            hparams = {
                HP_KERNEL_SIZE : kernel_size,
                HP_KERNEL_NUM : kernel_num,
                HP_LAMBDA_REG : lambda_reg
            }
            run_name = f'run-{session_num}'
            print('--- Starting Trial:',run_name)
            print({h.name: hparams[h] for h in hparams})
            run("Logs/Model 1 (Shoes_Male_L2)/hparam_tuning/" + run_name, hparams, session_num)

            session_num += 1

--- Starting Trial: run-1
{'kernel size': 3, 'kernel num': 64, 'lambda reg': 1e-05}
Epoch 1/15
65/65 [==============================] - 45s 695ms/step - loss: 1.1071 - accuracy: 0.6430 - sparse_categorical_crossentropy: 1.1012 - val_loss: 0.5299 - val_accuracy: 0.8108 - val_sparse_categorical_crossentropy: 0.5238
Epoch 2/15
65/65 [==============================] - 46s 701ms/step - loss: 0.5188 - accuracy: 0.8128 - sparse_categorical_crossentropy: 0.5125 - val_loss: 0.4210 - val_accuracy: 0.8243 - val_sparse_categorical_crossentropy: 0.4144
Epoch 3/15
65/65 [==============================] - 45s 691ms/step - loss: 0.4154 - accuracy: 0.8521 - sparse_categorical_crossentropy: 0.4086 - val_loss: 0.4794 - val_accuracy: 0.8108 - val_sparse_categorical_crossentropy: 0.4724
Epoch 4/15
65/65 [==============================] - 52s 793ms/step - loss: 0.3717 - accuracy: 0.8642 - sparse_categorical_crossentropy: 0.3644 - val_loss: 0.3940 - val_accuracy: 0.8417 - val_sparse_categorical_crossentropy:

Epoch 5/15
17/17 [==============================] - 4s 218ms/step - loss: 0.3788 - accuracy: 0.8591 - sparse_categorical_crossentropy: 0.3715
INFO:tensorflow:Assets written to: saved_models\Model 1 (Shoes_Male_L2)\Run-4\assets
--- Starting Trial: run-5
{'kernel size': 3, 'kernel num': 128, 'lambda reg': 0.0001}
Epoch 1/15
65/65 [==============================] - 112s 2s/step - loss: 1.3718 - accuracy: 0.6141 - sparse_categorical_crossentropy: 1.3128 - val_loss: 0.5743 - val_accuracy: 0.8243 - val_sparse_categorical_crossentropy: 0.5178
Epoch 2/15
65/65 [==============================] - 113s 2s/step - loss: 0.6017 - accuracy: 0.8022 - sparse_categorical_crossentropy: 0.5482 - val_loss: 0.5496 - val_accuracy: 0.8031 - val_sparse_categorical_crossentropy: 0.4987
Epoch 3/15
65/65 [==============================] - 115s 2s/step - loss: 0.5081 - accuracy: 0.8321 - sparse_categorical_crossentropy: 0.4581 - val_loss: 0.4744 - val_accuracy: 0.8263 - val_sparse_categorical_crossentropy: 0.4256


Epoch 4/15
65/65 [==============================] - 220s 3s/step - loss: 0.6410 - accuracy: 0.8285 - sparse_categorical_crossentropy: 0.4794 - val_loss: 0.7659 - val_accuracy: 0.7606 - val_sparse_categorical_crossentropy: 0.6108
Epoch 5/15
65/65 [==============================] - 207s 3s/step - loss: 0.5704 - accuracy: 0.8497 - sparse_categorical_crossentropy: 0.4209 - val_loss: 0.5635 - val_accuracy: 0.8456 - val_sparse_categorical_crossentropy: 0.4227
Epoch 6/15
65/65 [==============================] - 219s 3s/step - loss: 0.5250 - accuracy: 0.8591 - sparse_categorical_crossentropy: 0.3901 - val_loss: 0.5989 - val_accuracy: 0.8127 - val_sparse_categorical_crossentropy: 0.4683
Epoch 7/15
17/17 [==============================] - 6s 382ms/step - loss: 0.5635 - accuracy: 0.8456 - sparse_categorical_crossentropy: 0.4227
INFO:tensorflow:Assets written to: saved_models\Model 1 (Shoes_Male_L2)\Run-12\assets
